## Santa Barbara Climate Dashboard Showing Model Projections for Gridboxes in Santa Barbara County Extracted from Netcdf Files

#### Last time, I developed a skeleton dashboard showing campus point annual average minimum temperature data; however, the model projections weren't from the latest assessment report. For this week, I updated the dashboard to include data from Cal-Adapt using this download data tool link https://cal-adapt-3.vercel.app/dashboard/data-download-tool. This updated dashboard includes a dropdown menu for users to select a specific grid box in Santa Barbara with specific latitude/longitude coordinates. Once a grid box is selected, annual average minimum and maximum temperature line plots will be shown from 2015 to 2100 using the TaiESM1 model. The respective net CDF files shown in the code have data for daily minimum/maximum air temperatures; however, my code calculated the annual average for a specific year by averaging out the data points for each day in a year and dividing them by the number of years. My data shows annual average minimum/maximum temperature projections for each grid box from 2015 to 2100. 

#### Some gridboxes don't show data once they are selected since no data may be available from that gridbox and/or that gridbox encompasses the ocean. I also connected shiny to vscode to update the dashboard, and once the dashboard is fully developed, I can develop a web link that shows the dashboard from shiny for python. This week, I want to work on updating the dashboard to show every single climate variable as mentioned before, with different ssp scenario options/ for every single gridbox. So by the end of this week, I hope to develop a fully working mini climate dashboard for Santa Barbara. I also want to make the location data map more user friendly by turning the map gridboxes into clear, noticeable buttons for users to extract data from instead of a dropdown menu. I also want to include observational data for each climate variable like what is shown for the campus point. 

#### The following code extracts data from two netCDF files. The campus point data page is kept intact; however, the location data page has been updated to show a map of Santa Barbara County with all the grid boxes. Users can select a grid box and view plots for annual average minimum and maximum temperature projections from 2015 to 2100 for that specific grid box. 

In [5]:
from shiny import App, ui, render, reactive
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import xarray as xr
import os
from shinywidgets import output_widget, render_widget

# --- Constants and File Paths ---
# NetCDF file paths for minimum and maximum daily air surface temperature
NETCDF_FILE_TASMIN = "06083_tasmin_day_TaiESM1_ssp370_r1i1p1f1.nc"
NETCDF_FILE_TASMAX = "06083_tasmax_day_TaiESM1_ssp370_r1i1p1f1.nc"

# Latitude and Longitude bounds for the Santa Barbara region
SB_LAT_MIN, SB_LAT_MAX = 33.8, 35.2
SB_LON_MIN, SB_LON_MAX = -120.9, -119.3 # Note: NetCDF longitudes might be 0-360, adjust for -180 to 180 if needed

# Global variables to store loaded NetCDF datasets
netcdf_ds_tasmin = None
netcdf_ds_tasmax = None

# --- Load NetCDF datasets once globally ---
def load_netcdfs():
    """
    Loads the global NetCDF datasets for minimum and maximum temperature.
    This function is called once when the application starts to ensure
    the data is available for all sessions. It handles FileNotFoundError.
    """
    global netcdf_ds_tasmin, netcdf_ds_tasmax
    # Only load if they haven't been loaded already (e.g., on app restart)
    if netcdf_ds_tasmin is None or netcdf_ds_tasmax is None:
        try:
            netcdf_ds_tasmin = xr.open_dataset(NETCDF_FILE_TASMIN)
            netcdf_ds_tasmax = xr.open_dataset(NETCDF_FILE_TASMAX)
            print("NetCDF datasets loaded successfully.")
        except FileNotFoundError as e:
            print(f"ERROR: NetCDF file not found. Make sure '{e.filename}' is in the correct directory.")
            # Initialize as empty Datasets to prevent further errors
            netcdf_ds_tasmin = xr.Dataset()
            netcdf_ds_tasmax = xr.Dataset()
        except Exception as e:
            print(f"An unexpected error occurred while loading NetCDF files: {e}")
            netcdf_ds_tasmin = xr.Dataset()
            netcdf_ds_tasmax = xr.Dataset()

# Call the function to load NetCDF data when the script starts
load_netcdfs()

# --- Load CSV Data ---
def load_csv_minimum_temperature_data():
    """
    Loads minimum temperature data from a CSV file.
    It searches for 'chart.csv', 'chart (1).csv', or 'chart1.csv' in the current
    directory and its subdirectories. It attempts to read the CSV, skipping
    the first 8 rows if necessary, and renames columns by stripping whitespace.
    It also converts the 'year' column to numeric and handles missing values.
    """
    try:
        possible_filenames = ['chart.csv', 'chart (1).csv', 'chart1.csv']
        found_path = None
        # Walk through the current directory and its subdirectories to find the CSV
        for root, _, files in os.walk("."):
            for f in files:
                if f in possible_filenames:
                    found_path = os.path.join(root, f)
                    break
            if found_path:
                break
        if not found_path:
            raise FileNotFoundError("No CSV file matching expected names found.")
        
        # Attempt to read CSV, first skipping 8 rows, then without skipping
        try:
            df = pd.read_csv(found_path, skiprows=8)
        except Exception:
            df = pd.read_csv(found_path)

        # Clean column names by stripping whitespace
        df.rename(columns=lambda x: x.strip(), inplace=True)
        # Convert the first column to 'year' and ensure it's numeric
        df["year"] = pd.to_numeric(df.iloc[:, 0], errors="coerce")
        # Drop rows where 'year' is NaN (conversion failed)
        df.dropna(subset=["year"], inplace=True)
        df["year"] = df["year"].astype(int) # Convert 'year' to integer
        return df

    except Exception as e:
        print(f"Error loading CSV: {e}")
        return None

# --- Utility Functions for NetCDF Data ---
def find_nearest_gridpoint(lat_arr, lon_arr, target_lat, target_lon):
    """
    Finds the indices of the grid point nearest to the target latitude and longitude.
    Args:
        lat_arr (np.array): Array of latitudes from the dataset.
        lon_arr (np.array): Array of longitudes from the dataset.
        target_lat (float): The target latitude.
        target_lon (float): The target longitude (can be 0-360 or -180 to 180).
    Returns:
        tuple: (lat_idx, lon_idx) of the nearest grid point.
    """
    # Adjust target_lon to match the 0-360 range if lon_arr is in that range
    # Assuming lon_arr is primarily 0-360, convert target_lon if it's negative
    if np.any(lon_arr > 180) and target_lon < 0:
        target_lon += 360
    # If lon_arr is -180 to 180, and target_lon is > 180 (from dropdown), convert it
    elif np.any(lon_arr < 0) and target_lon > 180:
        target_lon -= 360

    lat_idx = (np.abs(lat_arr - target_lat)).argmin()
    lon_idx = (np.abs(lon_arr - target_lon)).argmin()
    return lat_idx, lon_idx

def extract_timeseries(ds, lat_idx, lon_idx, variable_name=None, years_limit=None):
    """
    Extracts a time series for a specific grid point from an xarray Dataset.
    Args:
        ds (xr.Dataset): The xarray Dataset.
        lat_idx (int): Latitude index of the grid point.
        lon_idx (int): Longitude index of the grid point.
        variable_name (str, optional): The name of the variable to extract.
                                       If None, attempts to extract the first variable.
        years_limit (int, optional): If provided, limits the data to the last N years
                                     from the current date.
    Returns:
        pd.DataFrame: A DataFrame with 'Date' and the extracted variable's values.
    """
    if "time" not in ds.coords:
        print("Error: 'time' coordinate not found in dataset.")
        return pd.DataFrame({"Date": [], "No Data": []})

    times = pd.to_datetime(ds["time"].values)
    
    time_mask = np.full(len(times), True)
    if years_limit is not None:
        max_date = pd.Timestamp.now()
        min_date = max_date - pd.Timedelta(days=365*years_limit)
        time_mask = (times >= min_date) & (times <= max_date)

    var_to_extract = None
    if variable_name and variable_name in ds.data_vars:
        var_to_extract = variable_name
    elif ds.data_vars:
        var_to_extract = list(ds.data_vars.keys())[0] # Fallback to first variable

    if var_to_extract is None:
        print("Error: No variable to extract found in dataset.")
        return pd.DataFrame({"Date": [], "No Data": []})

    # Check if the variable has the expected dimensions (time, lat, lon)
    if ds[var_to_extract].ndim < 3 or ds[var_to_extract].dims[0] != 'time':
        print(f"Warning: Variable '{var_to_extract}' does not have expected dimensions (time, lat, lon).")
        return pd.DataFrame({"Date": [], "No Data": []})

    # Extract values for the specific grid point
    values = ds[var_to_extract][:, lat_idx, lon_idx].values
    
    # Ensure values are numeric
    if np.issubdtype(values.dtype, np.number):
        values = values[time_mask]
    else:
        print(f"Warning: Variable '{var_to_extract}' contains non-numeric data.")
        return pd.DataFrame({"Date": [], "No Data": []})

    filtered_times = times[time_mask]
    if len(filtered_times) == 0 or len(values) == 0:
        print("No data points after filtering by time or for the selected grid point.")
        return pd.DataFrame({"Date": [], "No Data": []})
        
    df = pd.DataFrame({"Date": filtered_times, var_to_extract: values})
    return df

def get_gridbox_choices(ds, step=1):
    """
    Generates a list of gridbox choices (latitude, longitude strings)
    within the Santa Barbara bounds from an xarray Dataset.
    Args:
        ds (xr.Dataset): The xarray Dataset.
        step (int): Step size to sample grid points (e.g., 1 for all, 5 for every 5th).
    Returns:
        list: A list of strings like "lat,lon" for dropdown choices.
    """
    if not all(coord in ds.coords for coord in ["lat", "lon"]):
        print("Error: 'lat' or 'lon' coordinates not found in dataset for gridbox choices.")
        return []
        
    lats = ds["lat"].values
    lons = ds["lon"].values
    choices = []
    for i_lat in range(0, len(lats), step):
        lat = lats[i_lat]
        if not (SB_LAT_MIN <= lat <= SB_LAT_MAX):
            continue # Skip latitudes outside Santa Barbara bounds
        for i_lon in range(0, len(lons), step):
            lon = lons[i_lon]
            # Adjust longitude for checking against SB_LON_MIN/MAX if it's in 0-360 range
            lon_check = lon - 360 if lon > 180 else lon
            if not (SB_LON_MIN <= lon_check <= SB_LON_MAX):
                continue # Skip longitudes outside Santa Barbara bounds
            choices.append(f"{lat:.3f},{lon:.3f}")
    return choices

# --- UI Definition ---
app_ui = ui.page_fluid(
    # Custom CSS for styling the dashboard
    ui.tags.head(
        ui.tags.style("""
            body {
                font-family: 'Inter', sans-serif;
                background-color: #f8f9fa;
                color: #333;
            }
            .custom-card {
                background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                color: white;
                border-radius: 15px;
                padding: 20px;
                margin: 10px 0;
                box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            }
            .metric-card {
                background: white;
                border-radius: 10px;
                padding: 15px;
                margin: 10px 0;
                box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
                border-left: 4px solid #667eea;
            }
            .text-center {
                text-align: center;
            }
            .text-muted {
                color: #6c757d;
            }
            .container {
                max-width: 960px;
                margin: auto;
                padding: 20px;
                background-color: #fff;
                border-radius: 8px;
                box-shadow: 0 0 10px rgba(0,0,0,0.05);
                margin-top: 20px;
                margin-bottom: 20px;
            }
            h2 {
                font-size: 2.5rem;
                margin-bottom: 20px;
                color: #2c3e50;
                text-shadow: 1px 1px 2px rgba(0,0,0,0.1);
            }
            h3 {
                font-size: 2rem;
                margin-top: 20px;
                margin-bottom: 15px;
                color: #34495e;
            }
            h4 {
                font-size: 1.5rem;
                margin-top: 15px;
                margin-bottom: 10px;
                color: #34495e;
            }
            ul {
                list-style-type: disc;
                margin-left: 20px;
            }
            li {
                margin-bottom: 5px;
            }
            hr {
                border-top: 1px solid #eee;
                margin: 20px 0;
            }
            .shiny-input-container {
                margin-bottom: 15px;
            }
            .js-plotly-plot {
                margin-top: 20px;
                border: 1px solid #e0e0e0;
                border-radius: 8px;
                overflow: hidden;
            }
        """)
    ),
    # Main title of the dashboard
    ui.h2("🌊 Santa Barbara Climate Data Dashboard", class_="text-center"),
    # Data source information
    ui.div(
        ui.p("📊 Data Source: Cal-Adapt CSV & NetCDF", class_="text-center text-muted"),
        class_="data-source"
    ),
    # Navigation tabs for different sections of the dashboard
    ui.navset_tab(
        ui.nav_panel("🏠 Welcome",
            ui.div(
                ui.h3("Welcome!", class_="text-center"),
                ui.p("Explore Santa Barbara's climate using Cal-Adapt data.", class_="text-center"),
                class_="container"
            )
        ),
        ui.nav_panel("📚 User Guide / Glossary / About",
            ui.div(
                ui.h4("User Guide"),
                ui.tags.ul([
                    ui.tags.li("Navigate to 'Location Map with Dropdown' tab to explore NetCDF data."),
                    ui.tags.li("On the map, blue lines indicate grid cells. The red box outlines the Santa Barbara region."),
                    ui.tags.li("Use the 'Select a Gridbox' dropdown to choose a specific grid cell for detailed temperature plots."),
                    ui.tags.li("Navigate to 'Campus Point Data' tab to explore CSV data."),
                    ui.tags.li("Select 'Minimum/Maximum Temperature Projections' as Data Type."),
                    ui.tags.li("Select 'Annual Averages' as Temperature Data Type."),
                    ui.tags.li("Use the year range slider and column checkboxes to explore the CSV data."),
                ]),
                ui.hr(),
                ui.h4("Glossary"),
                ui.p("Cal-Adapt: California’s climate planning platform, providing climate change research and data."),
                ui.p("NetCDF: Network Common Data Form, a set of interfaces for array-oriented data."),
                ui.p("xarray: Python library for working with labeled multi-dimensional arrays, often used with NetCDF."),
                ui.p("tasmin: Daily minimum near-surface air temperature (from NetCDF)."),
                ui.p("tasmax: Daily maximum near-surface air temperature (from NetCDF)."),
                ui.p("SSP370: Shared Socioeconomic Pathway 3-7.0, a scenario representing a medium-to-high challenge to mitigation and adaptation."),
                ui.p("RCP 8.5: Representative Concentration Pathway 8.5, a high greenhouse gas emission scenario (from CSV)."),
                ui.p("RCP 4.5: Representative Concentration Pathway 4.5, an intermediate greenhouse gas emission scenario (from CSV)."),
                ui.p("CanESM2, CNRM-CM5, HadGEM2-ES, MIROC5: Different climate models used for projections."),
                class_="container"
            )
        ),
        ui.nav_panel("📍 Location Map with Dropdown",
            ui.div(
                ui.h3("Explore Climate Data by Grid Location"),
                ui.p("Select a gridbox on the map or from the dropdown to view detailed temperature projections."),
                output_widget("gridbox_map"), # Plotly mapbox for grid visualization
                ui.output_ui("gridbox_dropdown_ui"), # Dropdown for selecting gridbox
                ui.output_text("selected_gridbox_info"), # Displays info about selected gridbox
                ui.h4("Annual Average Minimum Air Surface Temperature (NetCDF)"),
                output_widget("selected_gridbox_tasmin_plot"), # Plot for tasmin data
                ui.h4("Annual Average Maximum Air Surface Temperature (NetCDF)"),
                output_widget("selected_gridbox_tasmax_plot"), # Plot for tasmax data
                class_="container"
            )
        ),
        # Campus Point Data panel (CSV strictly)
        ui.nav_panel("📊 Campus Point Data",
            ui.div(
                ui.h3("Campus Point Climate Data"),
                ui.p("This section visualizes historical and projected climate data specifically for Campus Point from CSV sources."),
                ui.input_select("location", "Select Location:", choices=["Campus Point"]),
                ui.input_select("data_type", "Select Data Type:", choices=[
                    "Minimum/Maximum Temperature Projections",
                    "Precipitation", "Sea Level Rise", "Droughts", "Wildfires"
                ]),
                ui.output_ui("temp_subcategory_ui"), # Dynamic UI for temperature subcategories
                ui.output_ui("graph_controls_ui"), # Dynamic UI for graph controls (slider, checkboxes)
                output_widget("climate_plot"), # Plotly plot for CSV data
                class_="container"
            )
        )
    )
)

# --- Server Logic ---
def server(input, output, session):
    # Reactive value to store the loaded CSV data
    csv_data = reactive.Value(None)
    # Reactive value to store the currently selected gridbox (lat, lon) from dropdown
    selected_gridbox = reactive.Value(None)
    # Reactive value for the SSP scenario (currently fixed to ssp370 as per file names)
    selected_ssp = reactive.Value("ssp370") # This could be made an input if multiple SSP files are available

    @reactive.Effect
    def load_csv_once():
        """
        Loads the CSV data once when the app starts or when `csv_data` is None.
        """
        if csv_data.get() is None:
            df = load_csv_minimum_temperature_data()
            csv_data.set(df)
            if df is None:
                print("Failed to load CSV data.")

    # --- Location Map Logic (NetCDF) -----------------------------------
    @output
    @render_widget
    def gridbox_map():
        """
        Renders an interactive Plotly Mapbox displaying grid points
        within the Santa Barbara region and the county boundary.
        """
        ds = netcdf_ds_tasmin # Use tasmin dataset for map grid points
        if ds is None or not all(coord in ds.coords for coord in ["lat", "lon"]):
            fig = go.Figure()
            fig.add_annotation(text="NetCDF files not loaded or missing coordinates. Please check server logs.", x=0.5, y=0.5, showarrow=False)
            return fig

        lats = ds["lat"].values
        lons_orig = ds["lon"].values # Original longitudes (can be 0-360)
        # Convert longitudes to -180 to 180 range for Plotly Mapbox
        lons_plot = np.where(lons_orig > 180, lons_orig - 360, lons_orig)

        # Calculate half-step for drawing grid cell boundaries
        delta_lat = (lats[1] - lats[0]) / 2 if len(lats) > 1 else 0.1
        delta_lon = (lons_plot[1] - lons_plot[0]) / 2 if len(lons_plot) > 1 else 0.1
        
        lat_list, lon_list, custom_data_list = [], [], []
        # Iterate through grid points to draw bounding boxes
        for i, lat in enumerate(lats):
            for j, lon_plot in enumerate(lons_plot):
                # Check if the grid point is within the Santa Barbara bounds
                original_lon_for_check = lons_orig[j]
                if original_lon_for_check > 180:
                    original_lon_for_check -= 360 # Adjust for check
                
                if not (SB_LAT_MIN <= lat <= SB_LAT_MAX and SB_LON_MIN <= original_lon_for_check <= SB_LON_MAX):
                    continue # Skip grid points outside the defined bounds
                
                # Define corners of the grid cell
                lat0, lat1 = lat - delta_lat, lat + delta_lat
                lon0, lon1 = lon_plot - delta_lon, lon_plot + delta_lon
                
                # Add points for the grid cell rectangle
                lat_list.extend([lat0, lat0, lat1, lat1, lat0, None])
                lon_list.extend([lon0, lon1, lon1, lon0, lon0, None])
                
                # Store original lat/lon for hover information
                custom_data_list.extend([(lat, lons_orig[j])] * 6)

        # Define Santa Barbara county boundary for visualization
        county_lat = [SB_LAT_MIN, SB_LAT_MIN, SB_LAT_MAX, SB_LAT_MAX, SB_LAT_MIN, None]
        county_lon = [SB_LON_MIN, SB_LON_MAX, SB_LON_MAX, SB_LON_MIN, SB_LON_MIN, None]

        fig = go.Figure()
        
        # Add grid lines trace
        fig.add_trace(go.Scattermapbox(
            lat=lat_list, 
            lon=lon_list, 
            mode="lines", 
            line=dict(width=1.5, color="blue"), 
            hoverinfo="text",
            customdata=custom_data_list,
            hovertemplate="<b>Gridbox:</b><br>Latitude: %{customdata[0]:.3f}<br>Longitude: %{customdata[1]:.3f}<extra></extra>",
            showlegend=False,
            name="Grid Cells"
        ))
        
        # Add Santa Barbara county boundary trace
        fig.add_trace(go.Scattermapbox(
            lat=county_lat, 
            lon=county_lon, 
            mode="lines", 
            line=dict(width=2, color="red"), 
            hoverinfo="skip", 
            showlegend=False, 
            opacity=0.7,
            name="SB County Boundary"
        ))
        
        # Update map layout
        fig.update_layout(
            mapbox_style="open-street-map", 
            mapbox_center={"lat": (SB_LAT_MIN + SB_LAT_MAX) / 2, "lon": (SB_LON_MIN + SB_LON_MAX) / 2}, 
            mapbox_zoom=8, 
            margin={"r":0,"t":0,"l":0,"b":0},
            height=500 # Set a fixed height for the map
        )
        return fig

    @output
    @render.ui
    def gridbox_dropdown_ui():
        """
        Renders a dropdown UI element with choices for gridboxes within the SB region.
        """
        ds = netcdf_ds_tasmin # Use tasmin dataset for grid choices
        choices = get_gridbox_choices(ds, step=1) # Get all gridbox choices
        return ui.input_select("gridbox_dropdown", "Select a Gridbox:", choices=choices, selected=choices[0] if choices else None)

    @reactive.Effect
    def update_selected_gridbox():
        """
        Updates the `selected_gridbox` reactive value when the dropdown selection changes.
        """
        sel = input.gridbox_dropdown()
        if sel:
            lat_str, lon_str = sel.split(",")
            selected_gridbox.set((float(lat_str), float(lon_str)))

    @output
    @render.text
    def selected_gridbox_info():
        """
        Displays the latitude and longitude of the currently selected gridbox.
        """
        val = selected_gridbox.get()
        if val:
            return f"Selected Gridbox: Latitude {val[0]:.3f}, Longitude {val[1]:.3f}"
        return "Select a gridbox from the dropdown."

    @reactive.Calc
    def _grid_indices():
        """
        Calculates the latitude and longitude indices for the selected gridbox
        from the NetCDF datasets.
        """
        val = selected_gridbox.get()
        if val is None or netcdf_ds_tasmin is None or netcdf_ds_tasmax is None or \
           not all(c in netcdf_ds_tasmin.coords for c in ["lat", "lon"]):
            return None, None, None, None # Removed the extra None from the return

        target_lat, target_lon_dropdown = val[0], val[1]
        lats_ds = netcdf_ds_tasmin["lat"].values
        lons_ds = netcdf_ds_tasmin["lon"].values

        lat_idx, lon_idx = find_nearest_gridpoint(lats_ds, lons_ds, target_lat, target_lon_dropdown)
        
        # Corrected return statement to include target_lat and target_lon_dropdown
        return lat_idx, lon_idx, target_lat, target_lon_dropdown

    @output
    @render_widget
    def selected_gridbox_tasmin_plot():
        """
        Renders a Plotly line plot of annual average minimum air surface temperature
        for the selected gridbox from the NetCDF data.
        Converts Kelvin to Fahrenheit if necessary.
        """
        lat_idx, lon_idx, target_lat, target_lon_dropdown = _grid_indices()
        ssp = selected_ssp.get()
        model = netcdf_ds_tasmin.attrs.get("source_id", "Unknown Model")

        fig = go.Figure()

        if lat_idx is None or lon_idx is None:
            fig.add_annotation(text="Select a gridbox to view data.", x=0.5, y=0.5, showarrow=False)
            return fig

        df_tasmin_raw = extract_timeseries(netcdf_ds_tasmin, lat_idx, lon_idx, variable_name="tasmin")
        tasmin_units_original = netcdf_ds_tasmin['tasmin'].attrs.get('units', 'Units Unknown')
        df_tasmin = df_tasmin_raw.copy()

        tasmin_display_units = tasmin_units_original
        # Convert Kelvin to Fahrenheit if units are Kelvin
        if "tasmin" in df_tasmin.columns and tasmin_units_original.lower() in ['k', 'kelvin']:
            df_tasmin["tasmin"] = (df_tasmin["tasmin"] - 273.15) * 9/5 + 32
            tasmin_display_units = '°F'

        is_tasmin_plottable = False
        if "tasmin" in df_tasmin.columns and not df_tasmin["tasmin"].isnull().all():
            df_tasmin.set_index("Date", inplace=True)
            # Resample to annual mean
            df_tasmin_annual = df_tasmin.resample('Y').mean().reset_index()
            df_tasmin_annual["Year"] = df_tasmin_annual["Date"].dt.year
            is_tasmin_plottable = True

        if is_tasmin_plottable:
            fig.add_trace(go.Scatter(x=df_tasmin_annual["Year"], y=df_tasmin_annual["tasmin"], mode="lines", name="Annual Avg Min Temp"))
            fig.update_layout(
                title=f"Annual Average Minimum Temperature ({model}, {ssp})",
                xaxis_title="Year",
                yaxis_title=f"Temperature ({tasmin_display_units})",
                height=400,
                showlegend=True,
                hovermode="x unified"
            )
        else:
            fig.add_annotation(
                text="No plottable data for Daily Minimum Air Surface Temperature.<br>(Point may be ocean or missing data.)",
                showarrow=False, xref="paper", yref="paper", x=0.5, y=0.5, align="center"
            )
            fig.update_layout(height=400)

        return fig

    @output
    @render_widget
    def selected_gridbox_tasmax_plot():
        """
        Renders a Plotly line plot of annual average maximum air surface temperature
        for the selected gridbox from the NetCDF data.
        Converts Kelvin to Fahrenheit if necessary.
        """
        lat_idx, lon_idx, target_lat, target_lon_dropdown = _grid_indices()
        ssp = selected_ssp.get()
        model = netcdf_ds_tasmax.attrs.get("source_id", "Unknown Model")

        fig = go.Figure()

        if lat_idx is None or lon_idx is None:
            fig.add_annotation(text="Select a gridbox to view data.", x=0.5, y=0.5, showarrow=False)
            return fig

        df_tasmax_raw = extract_timeseries(netcdf_ds_tasmax, lat_idx, lon_idx, variable_name="tasmax")
        tasmax_units_original = netcdf_ds_tasmax['tasmax'].attrs.get('units', 'Units Unknown')
        df_tasmax = df_tasmax_raw.copy()

        tasmax_display_units = tasmax_units_original
        # Convert Kelvin to Fahrenheit if units are Kelvin
        if "tasmax" in df_tasmax.columns and tasmax_units_original.lower() in ['k', 'kelvin']:
            df_tasmax["tasmax"] = (df_tasmax["tasmax"] - 273.15) * 9/5 + 32
            tasmax_display_units = '°F'

        is_tasmax_plottable = False
        if "tasmax" in df_tasmax.columns and not df_tasmax["tasmax"].isnull().all():
            df_tasmax.set_index("Date", inplace=True)
            # Resample to annual mean
            df_tasmax_annual = df_tasmax.resample('Y').mean().reset_index()
            df_tasmax_annual["Year"] = df_tasmax_annual["Date"].dt.year
            is_tasmax_plottable = True

        if is_tasmax_plottable:
            fig.add_trace(go.Scatter(x=df_tasmax_annual["Year"], y=df_tasmax_annual["tasmax"], mode="lines", name="Annual Avg Max Temp"))
            fig.update_layout(
                title=f"Annual Average Maximum Temperature ({model}, {ssp})",
                xaxis_title="Year",
                yaxis_title=f"Temperature ({tasmax_display_units})",
                height=400,
                showlegend=True,
                hovermode="x unified"
            )
        else:
            fig.add_annotation(
                text="No plottable data for Daily Maximum Air Surface Temperature.<br>(Point may be ocean or missing data.)",
                showarrow=False, xref="paper", yref="paper", x=0.5, y=0.5, align="center"
            )
            fig.update_layout(height=400)

        return fig

    # --- Campus Point Data UI Logic (CSV) ----------------------------
    @output
    @render.ui
    def temp_subcategory_ui():
        """
        Dynamically renders the 'Select Temperature Data Type' dropdown
        based on the main 'Data Type' selection.
        """
        if input.data_type() == "Minimum/Maximum Temperature Projections":
            return ui.input_select("temp_subcategory", "Select Temperature Data Type:", choices=[
                "Annual Averages",
                "Extreme Heat Days and Warm Nights",
                "Cooling Degree Days and Heating Degree Days"
            ])
        return None

    @output
    @render.ui
    def graph_controls_ui():
        """
        Dynamically renders the year range slider and column checkboxes
        based on 'Data Type' and 'Temperature Data Type' selections.
        This section has been updated to include all the specified model projections.
        """
        if input.data_type() == "Minimum/Maximum Temperature Projections" and input.temp_subcategory() == "Annual Averages":
            return ui.div(
                ui.input_slider("year_range", "Year Range:", min=1950, max=2100, value=(1950, 2006)),
                ui.input_checkbox_group(
                    "selected_columns", "Select Data to Display:",
                    choices=[
                        "Observed",
                        "Modeled RCP 8.5 Range Min",
                        "Modeled RCP 8.5 Range Max",
                        "CanESM2 (Average)",
                        "CNRM-CM5 (Cool/Wet)",
                        "HadGEM2-ES (Warm/Dry)",
                        "MIROC5 (Complement)"
                    ],
                    selected=["Observed"]
                )
            )
        return None

    @output
    @render_widget
    def climate_plot():
        """
        Renders the Plotly climate plot for Campus Point data (CSV).
        Displays a line plot for selected temperature projection columns.
        This function has been updated to directly check for the exact column names.
        """
        df = csv_data.get() # Get the loaded DataFrame
        if df is None or df.empty:
            fig = go.Figure()
            fig.add_annotation(text="CSV data not loaded or empty. Please check server logs.", x=0.5, y=0.5, showarrow=False)
            return fig

        # Plot for Minimum/Maximum Temperature Projections with Annual Averages
        if input.data_type() == "Minimum/Maximum Temperature Projections" and input.temp_subcategory() == "Annual Averages":
            year_min, year_max = input.year_range()
            selected_cols = input.selected_columns()
            if selected_cols is None:
                selected_cols = []

            filtered_df = df[(df["year"] >= year_min) & (df["year"] <= year_max)]
            fig = go.Figure()

            # Add a trace for each selected column, directly checking for exact column names
            for col in selected_cols:
                if col in filtered_df.columns: # Direct check for exact column name
                    fig.add_trace(go.Scatter(x=filtered_df["year"], y=filtered_df[col], mode="lines", name=col))
                else:
                    print(f"Warning: Column '{col}' not found in CSV data for plotting. Please ensure CSV column names match the selection options.")

            fig.update_layout(
                title=f"{input.data_type()} - {input.temp_subcategory()} at Campus Point",
                xaxis_title="Year",
                yaxis_title="Temperature (°F or as in CSV)",
                height=450,
                hovermode="x unified"
            )
            return fig

        # Fallback empty figure for other data types not yet implemented
        fig = go.Figure()
        fig.add_annotation(text="Data visualization for this data type not implemented yet.", x=0.5, y=0.5, showarrow=False)
        return fig

# Create the Shiny app instance
app = App(app_ui, server)

/var/folders/9y/cmg0_lnj3y1b07xwqw5nvmv80000gn/T/ipykernel_14153/901180878.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


ERROR: NetCDF file not found. Make sure '/Users/ozairusmani/Summer 2025 REU/06083_tasmin_day_TaiESM1_ssp370_r1i1p1f1.nc' is in the correct directory.
